In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import Dataframe

In [2]:
import pandas as pd
import yfinance
import numpy as np
import pandas_ta as pta

ticker = yfinance.Ticker("AAPL")
df = ticker.history(period = '5y')
df.drop(columns=['Dividends','Stock Splits'],inplace=True)

In [3]:
df['ema8']=pta.ema(df['Close'],length=8)
df['ema13']=pta.ema(df['Close'],length=13)
df['ema21']=pta.ema(df['Close'],length=21)
df['ema34']=pta.ema(df['Close'],length=34)
df['ema55']=pta.ema(df['Close'],length=55)
df['ema89']=pta.ema(df['Close'],length=89)
df['ema144']=pta.ema(df['Close'],length=144)
df['ema233']=pta.ema(df['Close'],length=233)

#Calculate MACD and Signal line
df['MACD_LINE']= pta.macd(df['Close'])['MACD_12_26_9']
df['MACD_SIGNAL']= pta.macd(df['Close'])['MACDs_12_26_9']

#Calculate RSI
df['RSI']=pta.rsi(df['Close'], length = 14)

#Calculate Stochastic(20,3)
df['STOCHK'] = pta.stoch(df['High'],df['Low'],df['Close'],k=20, d=3).STOCHk_20_3_3
df['STOCHD'] = pta.stoch(df['High'],df['Low'],df['Close'],k=20, d=3).STOCHd_20_3_3

#Calculate ROC(20)
df['ROC'] = pta.roc(df['Close'],length=20)

#Calculate CCI(20)
df['CCI'] = pta.cci(df['High'],df['Low'],df['Close'],length=20)

#Calculate Williamson%R(14)
df['WILLR'] = pta.willr(df['High'],df['Low'],df['Close'],length=14)

#Calculate MFI(14)
df['MFI'] = pta.mfi(df['High'],df['Low'],df['Close'],df['Volume'], length=14)

#Calculate ADX(14)
df['ADX'] = pta.adx(df['High'],df['Low'],df['Close'])['ADX_14']

#Calculate ATR(14)
df['ATR'] = pta.atr(df['High'],df['Low'],df['Close'],length=14)
df['ATR'] = pta.atr(df['High'],df['Low'],df['Close'],length=14)

#Calculate Bollinger Band(20,2)
df['BBL'] = pta.bbands(df['Close'],length=20)['BBL_20_2.0']
df['BBM'] = pta.bbands(df['Close'],length=20)['BBM_20_2.0']
df['BBH'] = pta.bbands(df['Close'],length=20)['BBU_20_2.0']

In [5]:
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    df = df.dropna(how='any')
    return df

In [23]:
df = clean_data(df)

In [24]:
df.isna().sum()

Open           0
High           0
Low            0
Close          0
Volume         0
ema8           0
ema13          0
ema21          0
ema34          0
ema55          0
ema89          0
ema144         0
ema233         0
MACD_LINE      0
MACD_SIGNAL    0
RSI            0
STOCHK         0
STOCHD         0
ROC            0
CCI            0
WILLR          0
MFI            0
ADX            0
ATR            0
BBL            0
BBM            0
BBH            0
TARGET         0
dtype: int64

In [25]:
df['TARGET'] = df['Close'].shift(-1)

In [26]:
df.tail()[['Close','TARGET']]

,Close,TARGET
Date,,
2022-02-11,168.639999,168.880005
2022-02-14,168.880005,172.789993
2022-02-15,172.789993,172.550003
2022-02-16,172.550003,168.880005
2022-02-17,168.880005,NaN


In [27]:
X=df.drop(columns=['TARGET'])
y=df['TARGET']

# Split Dataframe

In [30]:
len_ = int(0.8*df.shape[0])
df_train = df[:len_]
df_test = df[len_:]

In [33]:
len_ = int(0.8*df_train.shape[0])
df_train_1 = df[:len_]
df_val = df[len_:]

In [34]:
len(df_train)
X_train = df_train_1.drop(columns=['TARGET'])
y_train = df_train_1['TARGET']
X_val = df_val.drop(columns=['TARGET'])
y_val = df_val['TARGET']
X_test = df_test.drop(columns=['TARGET'])
y_test = df_test['TARGET']

# Create LSTM Model

In [37]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector

In [38]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization

def init_model():
    
    metric = metrics.MAPE
    opt = optimizers.RMSprop(learning_rate=0.001)
    normalizer = Normalization()
    normalizer.adapt(X_train)
    
    model = models.Sequential()
    model.add(normalizer)
    model.add(layers.LSTM(50, return_sequences=True, activation='relu',input_shape=(X_train.shape[-1],)))
    model.add(layers.LSTM(50, activation='tanh'))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    
    model.compile(loss='mse', 
                  optimizer=opt, 
                  metrics=[metric])
    
    return model


In [39]:
model = init_model()
model.summary()

ValueError: Input 0 of layer lstm_1 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 27)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train,
            validation_split=0.3,
            epochs=1000, 
            batch_size=32,
            callbacks=[es], verbose=0)

# Plot History

In [ ]:
plt.plot(history.history['mean_absolute_percentage_error'])
plt.plot(history.history['val_mean_absolute_percentage_error'])

# Test Model

In [ ]:
res = model_2.evaluate(X_test, y_test, verbose=0)

print(f'MAPE on the test set : {res[1]:.0f} %')